In [ ]:
import numpy as np 
import pandas as pd 
import os
import torch
!pip install timm
import torch.nn as nn
import timm
from torch.utils.data import DataLoader, Dataset
import cv2
import os
!pip install albumentations
import albumentations as A


scaler = torch.cuda.amp.GradScaler()
torch.cuda.is_available()
torch.manual_seed(1337)

     |████████████████████████████████| 376 kB 5.4 MB/s 
     |████████████████████████████████| 631 kB 5.2 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=386a23aa1e2ede750662cc6759e46fad5a2c3b93139687050e0c7097ca5c0072
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


!unzip /content/drive/MyDrive/prepared_data
! ls

In [ ]:
! diff /content/drive/MyDrive/train_giant.csv /content/images_train_resized/train_giant.csv

In [ ]:
#data_anno_path = '/kaggle/input/hogweed-dataset/'
data_anno_path = '/content/images_train_resized/'
old_test = pd.read_csv(data_anno_path+"train.csv")
df = pd.read_csv(data_anno_path+"train_giant.csv")#old_test.append(pd.read_csv(data_anno_path+"train_small.csv"))
print(df.shape)
df = df.drop(df[df.id == '4b58ef45756f7998b127952202a903e5'].index)
df = df.drop(df[df.id == '5a4e59e53496bdfdca7afed12f3afd9a'].index)
df = df.drop(df[df.id == '766e7669a5c7bb5f758acfd58ec05e0f'].index)
df.shape

(16497, 3)


(16494, 3)

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df,
                                     test_size=0.3,
                                     random_state=1337,
                                     shuffle=True,
                                     stratify=df['has_hogweed'])

print(len(train), len(validation), len(old_test))

11545 4949 311


In [ ]:

for name in df['id']:
 image_path1 = os.path.join(data_anno_path + 'has_hogweed/', name)
 image_path2 = os.path.join(data_anno_path + 'no_hogweed/', name)
 image_path3 = os.path.join(data_anno_path + 'giant_hogweed/', name)
 image_path4 = os.path.join(data_anno_path + 'giant_no_hogweed/', name)
 img = cv2.imread(image_path1 + '.jpg')
 if str(type(img)) == "<class 'NoneType'>":
   img = cv2.imread(image_path2 + '.jpg')
 if str(type(img)) == "<class 'NoneType'>":
   img = cv2.imread(image_path3 + '.jpg')
 if str(type(img)) == "<class 'NoneType'>":
   img = cv2.imread(image_path4 + '.jpg')  
 if str(type(img)) == "<class 'NoneType'>":
     print(name)
print('done')

done


# DATALOADER + AUGMENTATION

In [ ]:
class ClassifficationDataset(Dataset):
    def __init__(self, df, data_path, transrofmation=None, test=False):
        self.data_anno = df
        #self.data_path = data_path + 'images_small' if not test else data_path + 'images_test_resized/images_test_resized/unknown/'
        self.data_path = data_path
        self.transform = transrofmation
        self.test = test

    def __getitem__(self, idx):

        image_name = self.data_anno['id'].iloc[idx]
        if not self.test:
            label = self.data_anno['has_hogweed'].iloc[idx]
        if not self.test:
            image_path1 = os.path.join(self.data_path + 'has_hogweed/', image_name)
            image_path2 = os.path.join(self.data_path + 'no_hogweed/', image_name)
            image_path3 = os.path.join(self.data_path + 'giant_hogweed/', image_name)
            image_path4 = os.path.join(self.data_path + 'giant_no_hogweed/', image_name)
            img = cv2.imread(image_path1 + '.jpg')
            if str(type(img)) == "<class 'NoneType'>":
                img = cv2.imread(image_path2 + '.jpg')
            if str(type(img)) == "<class 'NoneType'>":
                img = cv2.imread(image_path3 + '.jpg')
            if str(type(img)) == "<class 'NoneType'>":
                img = cv2.imread(image_path4 + '.jpg')
        else:
            image_path = os.path.join(self.data_path, image_name)
            img = cv2.imread(image_path + '.jpg')
            
        if self.transform is not None:
            img = self.transform(image=img)["image"]

        img = self.preprocess(img)
        if self.test:
            return torch.from_numpy(img).float()
        else:
            return torch.from_numpy(img).float(), label

    def __len__(self):
        return len(self.data_anno)

    def preprocess(self, img):

        img = cv2.resize(img, (384, 384))
        img = img.transpose(2, 0, 1)
        img = (img - 127.5) / 128

        return img

In [ ]:
transformator = A.Compose([
#   A.Blur(p = 0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3),
#    A.HorizontalFlip(p=0.3),
#    A.ToGray(p=1)
    A.RandomBrightnessContrast(contrast_limit = 0.1, brightness_limit=0.1,p=0.5),
#    A.RandomFog(p=0.3),
#   A.ToGray(p=0.1)
])


In [ ]:



train_dataset = ClassifficationDataset(train,
                                      data_anno_path, 
                                      transrofmation=transformator)

valid_dataset = ClassifficationDataset(validation,
                                      data_anno_path, 
                                      transrofmation=None)

old_dataset = ClassifficationDataset(old_test,
                                      data_anno_path, 
                                      transrofmation=None)

train_dataloader = DataLoader(train_dataset, batch_size=15, shuffle=True,
                              num_workers=8)

val_dataloader = DataLoader(valid_dataset, batch_size=15, shuffle=False,
                            num_workers=8)

old_dataloader = DataLoader(old_dataset, batch_size=15, shuffle=False, 
                            num_workers=8)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
train_dataloader.__iter__().__next__()[0].shape

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([15, 3, 384, 384])

# MODEL GENERATING

In [ ]:

model = timm.create_model('efficientnet_b1', pretrained=True)
#model.fc = nn.Sequential(nn.Linear(2048, 2))
#model.head = nn.Linear(384, 6)
model.classifier = nn.Sequential(nn.Linear(1280, 2))
print(model)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
   

In [ ]:
#for p in model.parameters():
#    p.require_grad = False
#model.classifier = nn.Sequential(nn.Linear(1280, 2))
#model.fc = nn.Sequential(nn.Linear(2048, 2))


In [ ]:
model.cuda()
pass

# OPTIMIZER, SCHEDULER, ETC

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 2, eta_min = 1e-5)


# TRAIN + VAL

In [ ]:
num_epoch = 2
print_frec = 30

for epoch in range(num_epoch):
    lr_sched.step(epoch)
    total_loss = 0
    corrects = 0
    num_samples = 0
    model.train()

    for i, (img, label) in enumerate(train_dataloader):
        i += 1
        img = img.cuda()
        label = label.cuda()
        out = model(img)
        optimizer.zero_grad()
        #with torch.cuda.amp.autocast():
        #    loss = criterion(out, label)
        #scaler.scale(loss).backward()
        #scaler.step(optimizer)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_samples += img.size(0)
        _, preds = torch.max(out, 1)
        corrects += torch.sum(preds == label.data).item()
        #scaler.update()

        # if i % print_frec == 0:
        #     print('Loss:', total_loss / i, 'Train_accuracy:', corrects / num_samples)

    print('Epoch:',epoch,'Train_Loss:', total_loss / i, 'Train_accuracy:', corrects / num_samples)

    model.eval()
    val_correct = 0
    val_loss = 0
    num_samples = 0
    for i, (img, label) in enumerate(val_dataloader):
        img = img.cuda()
        label = label.cuda()
        out = model(img)
        #with torch.cuda.amp.autocast():
        #    loss = criterion(out, label)
        loss = criterion(out, label)
        val_loss += loss.item()
        num_samples += img.size(0)
        torch.save(model.state_dict(), '/content/drive/MyDrive/best_model.pth')

        _, preds = torch.max(out, 1)
        val_correct += torch.sum(preds == label.data).item()
    print('Val_Loss:', val_loss / i, 'Val_accuracy:', val_correct / num_samples,'LR: ', get_lr(optimizer))


torch.save(model.state_dict(), '/content/drive/MyDrive/best_model.pth')


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

RuntimeError: ignored

# Check old train accuracy

In [ ]:
torch.save(model.state_dict(), 'best_model.pth')

In [ ]:
num_epoch = 1
print_frec = 30

model.load_state_dict(torch.load('/content/drive/MyDrive/best_model.pth'))
model.eval()

for epoch in range(num_epoch):
    val_correct = 0
    val_loss = 0
    num_samples = 0
    for i, (img, label) in enumerate(old_dataloader):
        img = img.cuda()
        label = label.cuda()
        out = model(img)
        #with torch.cuda.amp.autocast():
        #    loss = criterion(out, label)
        loss = criterion(out, label)
        val_loss += loss.item()
        num_samples += img.size(0)

        _, preds = torch.max(out, 1)
        val_correct += torch.sum(preds == label.data).item()
    print('Val_Loss:', val_loss / i, 'Val_accuracy:', val_correct / num_samples)


# FINE-TUNE

preparing


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/best_model.pth'))
#for p in model.parameters():
#    p.require_grad = False

#model.classifier = nn.Sequential(nn.Linear(1280, 2))
model.cuda()
pass

In [ ]:
from sklearn.model_selection import train_test_split
tune, val = train_test_split(old_test,
                                     test_size=0.3,
                                     random_state=1337,
                                     shuffle=True,
                                     stratify=old_test['has_hogweed'])

print(len(tune), len(val))


In [ ]:
sum(val['has_hogweed'])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()
lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 3, eta_min = 1e-5)


In [ ]:
tune_dataset = ClassifficationDataset(tune,
                                      data_anno_path,test=False)
vaal_dataset = ClassifficationDataset(tune,
                                      data_anno_path,test=False)


tune_dataloader = DataLoader(tune_dataset, batch_size=15, shuffle=True,
                              num_workers=4)
vaal_dataloader = DataLoader(vaal_dataset, batch_size=15, shuffle=False,
                              num_workers=4)


In [ ]:
num_epoch = 3
print_frec = 30

for epoch in range(num_epoch):
    lr_sched.step(epoch)
    total_loss = 0
    corrects = 0
    num_samples = 0
    model.train()

    for i, (img, label) in enumerate(tune_dataloader):
        i += 1
        img = img.cuda()
        label = label.cuda()
        out = model(img)
        optimizer.zero_grad()
        #with torch.cuda.amp.autocast():
        #    loss = criterion(out, label)
        #scaler.scale(loss).backward()
        #scaler.step(optimizer)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_samples += img.size(0)
        _, preds = torch.max(out, 1)
        corrects += torch.sum(preds == label.data).item()
        #scaler.update()

        # if i % print_frec == 0:
        #     print('Loss:', total_loss / i, 'Train_accuracy:', corrects / num_samples)

    print('Epoch:',epoch,'Train_Loss:', total_loss / i, 'Train_accuracy:', corrects / num_samples)

    model.eval()
    val_correct = 0
    val_loss = 0
    num_samples = 0
    for i, (img, label) in enumerate(vaal_dataloader):
        img = img.cuda()
        label = label.cuda()
        out = model(img)
        #with torch.cuda.amp.autocast():
        #    loss = criterion(out, label)
        loss = criterion(out, label)
        val_loss += loss.item()
        num_samples += img.size(0)

        _, preds = torch.max(out, 1)
        val_correct += torch.sum(preds == label.data).item()
    print('Val_Loss:', val_loss / i, 'Val_accuracy:', val_correct / num_samples,'LR: ', get_lr(optimizer))


torch.save(model.state_dict(), 'best_model.pth')


# SUBMISSION

In [ ]:
torch.cuda.empty_cache() 

test_data_anno_path = '/content/images_test_resized/unknown'
test = pd.read_csv(test_data_anno_path + '/sample_submission.csv')
#print(test.head())
test_dataset = ClassifficationDataset(test,
                                      '/content/images_test_resized/unknown',test=True)

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False,
                              num_workers=4)


In [ ]:

model.load_state_dict(torch.load('best_model.pth'))
model.eval()
predicts = []
for img in test_dataloader:
    img = img.cuda()

    out = model(img)

    _, preds = torch.max(out, 1)
#     print(preds)
    predicts.append(preds.detach().cpu().numpy())
predicts = np.array(predicts)
predicts = np.concatenate(predicts)
test['has_hogweed'] = predicts
print(test)
test.to_csv('my_submission.csv', index=False)
